# Minio - файловое хранилище корпоративного уровня

[песочница](https://play.min.io/minio/login)

[документация](https://docs.minio.io/docs/python-client-api-reference)

Если пользуетесь песочницей - сохраните логин и пароль, которые вам даются при входе, по ним можно входить из кода.

Если пользуетесь Докером - при запуске он пишет код доступа и секретный ключ.

In [1]:
# Просто попробуем что-то сделать в песочнице Minio.
from minio import Minio
from minio.error import ResponseError

minioClient = Minio('play.min.io',
                  access_key='Q3AM3UQ867SPQQA43P2F',
                  secret_key='zuf+tfteSlswRu7BJ86wekitnifILbZam1KYY3TG',
                  secure=True)

In [2]:
# Список бакетов (каталогов) верхнего уровня.
buckets = minioClient.list_buckets()

In [6]:
# Что есть у бакета и сколько их в песочнице.
#dir(buckets[0])
# Если убрать len - получится список имен кааталогов. Каталоги могут быть вложенными.
len([b.name for b in buckets])

414

In [7]:
buckets[:5]

In [8]:
# Получаем список объектов (файлов) в бакете (каталоге).
objs = list(minioClient.list_objects_v2("deadbeef"))

In [9]:
# Что есть у объекта?
dir(objs[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'bucket_name',
 'content_type',
 'delete_marker',
 'etag',
 'is_dir',
 'is_latest',
 'last_modified',
 'metadata',
 'object_name',
 'owner_id',
 'owner_name',
 'size',
 'storage_class',
 'version_id']

In [10]:
# Какое у файла имя?
objs[0].object_name

'000506a2-20b5-5b83-be12-d95cc2aa5862.jpg'

In [11]:
# Получаем объект (файл) из хранилища как набор байтиков.
res = minioClient.get_object(objs[0].bucket_name, objs[0].object_name)

In [12]:
type(res)

urllib3.response.HTTPResponse

In [13]:
# Познакомьтесь, pdf-файл.
res.data[:1000]

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xe2\x02(ICC_PROFILE\x00\x01\x01\x00\x00\x02\x18\x00\x00\x00\x00\x02\x10\x00\x00mntrRGB XYZ \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00acsp\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\xf6\xd6\x00\x01\x00\x00\x00\x00\xd3-\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\tdesc\x00\x00\x00\xf0\x00\x00\x00trXYZ\x00\x00\x01d\x00\x00\x00\x14gXYZ\x00\x00\x01x\x00\x00\x00\x14bXYZ\x00\x00\x01\x8c\x00\x00\x00\x14rTRC\x00\x00\x01\xa0\x00\x00\x00(gTRC\x00\x00\x01\xa0\x00\x00\x00(bTRC\x00\x00\x01\xa0\x00\x00\x00(wtpt\x00\x00\x01\xc8\x00\x00\x00\x14cprt\x00\x00\x01\xdc\x00\x00\x00<mluc\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x0cenUS\x00\x00\x00X\x00\x00\x00\x1c\x00s\x00R\x00G\x00B\x00\x00\x0

In [14]:
# Получаем объект и сохраняем его сразу в файл.
res2 = minioClient.fget_object(objs[0].bucket_name, objs[0].object_name, "1.jpg")

In [15]:
minioClient.fput_object(objs[0].bucket_name, "2.pdf", "1.pdf")

('114e190dba6450b6988a7408333e927f', None)

In [36]:
# Можно получить кусочек файла, если он очень большой.
res_p = minioClient.get_partial_object("aio-ato-art", "2.pdf", 100, 100)
res_p.data

b'Marked true>>>>\r\nendobj\r\n2 0 obj\r\n<</Type/Pages/Count 1/Kids[ 3 0 R] >>\r\nendobj\r\n3 0 obj\r\n<</Type/Pa'

In [37]:
# Политики безопасности (не прописаны).
policy = minioClient.get_bucket_policy('aio-ato-art')
print(policy)

NoSuchBucketPolicy: NoSuchBucketPolicy: message: The specified bucket does not have a bucket policy.

Сделаем какую-то задачу, похожую на практику.

Скачиваем Ленту, новости кладем в хранилище. Дальше можно сделать как в работе по Редису - параллельные потоки, которые размечают эти тексты и сохраняют в файл.

In [17]:
import requests
#import pymorphy2
import re
from bs4 import BeautifulSoup
import datetime
import time
import io

In [38]:
# Функция для загрузки одной новости из Ленты.ру
def getLentaArticle(url):
    """ getLentaArticle gets the body of the article from Lenta.ru"""
    print("downloading lenta:", url)
    r = requests.get(url)
    p = BeautifulSoup(r.text, "html5")("p")
    if len(p) == 0:
        return "  "
    p = " ".join([a.get_text() for a in p])
    return p

# Загрузка новостей из Ленты.ру за некоторый период.
def getLenta():
    curdate = datetime.date(2017, 1, 16)
    findate = datetime.date(2017, 1, 16)
    res = ""
    
    fileno = 1
# Загружаем новости до конечной даты.
    while curdate <= findate:
        print('lenta ' + curdate.strftime('%Y/%m/%d'))
        day = requests.get('https://lenta.ru/news/' + curdate.strftime('%Y/%m/%d'))
        body = re.findall('<h3>(.+?)</h3>', day.text)
        links = ['https://lenta.ru' + re.findall('"(.+?)"', x)[0] for x in body]
        for l in links: # идем по всем ссылкам на новости за день.
            news = getLentaArticle(l)
            if news != "  ":
                # Преобразуем текст к набору байт.
                news = news.encode()
                news_len = len(news)
                f = io.BytesIO(news)
                # Кладем их в Minio.
                minioClient.put_object('aio-ato-art', "lenta"+str(fileno)+".txt", f, news_len, content_type='text/plain')
                fileno += 1
                if fileno>10:
                    break
            time.sleep(0.2)
        curdate += datetime.timedelta(days=1)
    print("lenta finished")


In [39]:
# Запускаем загрузку новостей.
getLenta()

lenta 2017/01/16
downloading lenta: https://lenta.ru/news/2017/01/16/apollo/
downloading lenta: https://lenta.ru/news/2017/01/16/kadyrov/
downloading lenta: https://lenta.ru/news/2017/01/16/trump2/
downloading lenta: https://lenta.ru/news/2017/01/16/accordion/
downloading lenta: https://lenta.ru/news/2017/01/16/trump3/
downloading lenta: https://lenta.ru/news/2017/01/16/bishkek4/
downloading lenta: https://lenta.ru/news/2017/01/16/bishkek5/
downloading lenta: https://lenta.ru/news/2017/01/16/realsevilla/
downloading lenta: https://lenta.ru/news/2017/01/16/safetymoscow/
downloading lenta: https://lenta.ru/news/2017/01/16/vertols/
lenta finished


<div style="background-color:#F00">А еще можно рассказать про Celery, <a href="https://habr.com/ru/company/itsumma/blog/416629/">RabbitMQ</a>, Heroku, больше про виртуализацию и Docker.</div>